In [1]:
import esprima as esp
import re 
from platform import python_version

print(python_version())

3.7.6


# 1. Extract Sequence of syntax units from Abstract Tree

In [68]:
# convert java script code to the abstract syntax tree
# then extract the sequences of syntax units with detailed information
i = 0
def sequence_of_syntax_units(scripts):
    global i
    i+=1
    print(i)
    """
    sctipts : list or single javascript code 
    return sequence of syntax units
    each syntax unit correspond to line in an abstract syntax tree
    """
    if type(scripts) is list:
        # esp.parseScript(script) returns abstract syntax tree of each js scripts
        return [re.sub('\s+', ' ', ''.join(str(esp.parseScript(script)).split('\n'))) for script in scripts]
    elif type(scripts) is str:
        return re.sub('\s+', ' ', ''.join(str(esp.parseScript(scripts)).split('\n')))
    else:
        raise ValueError('The type of scripts parameter must be {list or string}')

In [3]:
# test sequence_of_syntax_units 
test_script = """
alert(1);
"""
sequence_of_syntax_units(test_script)

'{ type: "Program", sourceType: "script", body: [ { type: "ExpressionStatement", expression: { type: "CallExpression", callee: { type: "Identifier", name: "alert" }, arguments: [ { type: "Literal", value: 1, raw: "1" } ] } } ]}'

## 2. Static WordEmedding

In [4]:
import gensim 
from nltk.tokenize import word_tokenize
import pickle 



############################
MODEL_FILENAME= 'static_word2vec.txt'
TRAIN_DATA_FILENAME= 'train_data.pkl'
MAX_LENGTH= 1024
EMBEDDING_DIM=50
############################

# create word tokens
def sequence_of_syntax_units_to_tokens(seq_of_syn_units):
    """
    Convert Sequence of syntax units to tokens to train  word2vec model 
    -----------------------------------------
    seq_of_syn_units: list of sequence syntax units correspond to abstract syntax trees
    """
    return list(map(word_tokenize, seq_of_syn_units))
        
    

# train static word2vec model
def train_static_word2vec(X_train, embedding_dim=50, window=5, min_count=1):
    """
    Train static word2vec model
    ----------------------------------------
    X_train: List of sequence syntax tree obtained from each script
    embedding dim:  The number of dimensions to represent each word
    window: Only terms that occur within a window-neighborhood of a term, in a sentence, are associated with it during training
    min_count:  Word with frequency greater than min_count only are going to be included into the model
    ---------------------------------------
    return word2vec model, vocabulary size
    """
    X_train_to_tokens = sequence_of_syntax_units_to_tokens(X_train)
    statc_word2vec_model = gensim.models.Word2Vec(sentences=X_train_to_tokens, size=embedding_dim, window=window, min_count=min_count)
    vocab_size = len(list(statc_word2vec_model.wv.vocab))
    print('Vocabulary Size : {}'.format(vocab_size))
    return statc_word2vec_model, vocab_size


# save model 
def save_static_word2vec(word2vec_model, model_filename, train_data, train_data_filename):
    """
    Save the trained word2vec model to the disk and the data which the model has been trained on it
    -----------------------------------------------------
    word2vec_model: model we want to save 
    model_filename: model file name and the path where to store the model 
    train_data: the data which  which the model has been trained on it (X_train scripts)
    train_data_filename: where to store the train_data and the file name 
    """
    word2vec_model.wv.save_word2vec_format(model_filename, binary=False)
    with open(train_data_filename, 'wb') as file:
        pickle.dump(train_data, file)
    return 

## 2.1 Use pre-trained Embedding 

In [5]:
import numpy as np

from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import warnings
warnings.filterwarnings('ignore')



def load_word2vec(model_name=MODEL_FILENAME, train_data_filename=TRAIN_DATA_FILENAME, max_len=MAX_LENGTH, embedding_dim=EMBEDDING_DIM):
    """
    Load the pre-trained static word2vec model
    -----------------------------------------
    model_name: path and file name of the model 
    train_data_filename: path and file name of the data used to train the model 
    max_len: max length used in padding sequence 
    embedding_dim: output shape of embedding model 
    
    -------------------------------------
    return embedding matrix , vocabulary size, embedding dim, max length
    """
    embedding_index = {}
    with open(model_name, 'r') as file:
        for line in file:
            values = line.split()
            word, vec = values[0], np.asarray(values[1:])
            embedding_index[word] = vec
    # vectorize the text samples into a 2D integer tensor
    with open(train_data_filename, 'rb') as file:
        train_data = pickle.load(file)
    tokenizer = Tokenizer(num_words=400000)
    tokenizer.fit_on_texts(train_data)
    sequences = tokenizer.texts_to_sequences(train_data)

    #pad sequences
    word_index = tokenizer.word_index
    sequences_pad = pad_sequences(sequences, maxlen=max_len, padding='post')
    
    vocab_size = len(word_index) + 1
    embedding_matrix = np.zeros((vocab_size, embedding_dim)) # number of unique words * embedding dim
    for word, i in word_index.items():
        if i > vocab_size:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix, vocab_size, embedding_dim, max_len

# 3. Dynamic WordEmbedding using BiLSTM

In [6]:
def preprocessing_dataset(X_train, X_test, max_len=MAX_LENGTH):
    """
    Map tokens to integers to feed them into Embedding layer
    """
    tokenizer = Tokenizer(400000)
    tokenizer.fit_on_texts(X_train)
    X_train_sequence = tokenizer.texts_to_sequences(X_train)
    X_test_sequence = tokenizer.texts_to_sequences(X_test)
    return pad_sequences(X_train_sequence, maxlen=max_len, padding='post'), pad_sequences(X_test_sequence, maxlen=max_len, padding='post')

In [7]:
from keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout, Conv1D, GlobalMaxPooling1D
from keras.initializers import Constant
from keras.models import Sequential
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.utils import plot_model
from keras import Model
import pydot

def neural_network():
    """
    Model Architecture :
    Static WordEmbedding using keras Embedding layer with trainable parameter equal flase
    daynamic WordEmbedding using 2 bi-lstm layers 
    TextCNN with 4 conv layers
    """
    model = Sequential()
    # static embedding layer 
    static_word2vec_matrix, vocab_size, embedding_dim, max_len = load_word2vec()
    
    model.add(Embedding(vocab_size,
                       embedding_dim,
                       embeddings_initializer=Constant(static_word2vec_matrix), 
                       input_length=max_len,
                       trainable=False
                       )
             )
    
    
    # dynamic word2vec using 2 bi-lstm layers
    model.add(Bidirectional(LSTM(units=50, input_shape=(None, 50), return_sequences=True)))
    model.add(Bidirectional(LSTM(units=50, input_shape=(None, 50), return_sequences=True)))
    
    # TextCNN with 4 conv layers
    model.add(Conv1D(128, 7, activation='tanh', input_shape=(None, 32)))
    model.add(Conv1D(128, 15, activation='tanh'))
    model.add(Conv1D(128, 25, activation='tanh'))
    model.add(Conv1D(128, 35, activation='tanh'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    # hyperparameters 
    adam = Adam(learning_rate=0.01)
    model.compile(
        loss=BinaryCrossentropy(from_logits=True), 
        optimizer=adam, 
        metrics=['accuracy']
    )
    model.summary()
    return model

# JSContana pipeline

In [8]:
# m = neural_network()

In [9]:
def pipeline(scripts, target, test_size=0.2):
    """
    scripts : train and test dataset
    target : malicious file or not 
    test_size: test set size
    """
    # 1- convert each script in X to abstract tree then to sequence of syntax units
    X = sequence_of_syntax_units(scripts)
    
    # 2- train test split 
    indeices = np.arange(len(scripts)).astype('int')
    np.random.shuffle(indeices)
    X = np.array(X)
    target = np.array(target)
    X = X[indeices]
    y = target[indeices]
    num_of_test_samples = int(test_size * len(scripts))
    X_train, X_test, y_train, y_test = X[:-num_of_test_samples], X[-num_of_test_samples:], y[:-num_of_test_samples], y[-num_of_test_samples:]
    
    
    # 3- train static word2vec model and save it 
    model, voc_size = train_static_word2vec(X_train)
    save_static_word2vec(model, MODEL_FILENAME, X_train, TRAIN_DATA_FILENAME)
    
    # 4- build nerual network
    nn = neural_network()
    # preprocess input
    X_train_, X_test_ = preprocessing_dataset(X_train, X_test)
    # feed data to nn
    
    history = nn.fit(X_train_, y_train, epochs=1, validation_split=0.1, verbose=True, batch_size=32)
    return nn, history

In [10]:
def extract_js_code_with_no_tag(code):
    return  list(map(lambda x: re.sub(r';', ' ; ', x), re.findall(r'javascript:(.*?\(.*?\))[\\/<>]?', code, re.IGNORECASE)))
    
def extract_js_code(script):

    #list all the scripts tags
    internal_js = re.findall(r'<script>?(.*?\(.*?\))[\\/<>]?',script , re.IGNORECASE)
    # remove unwanted words like javascript keyword and quotes 
    # add space befor and after semicolon
    clean_js_codes = []
    for code in internal_js:
        if re.search(r'javascript', code):
            clean_js_codes += extract_js_code_with_no_tag(code)  
        else:
            clean_js_codes.append(re.sub(r';', ' ; ', code))
    r = list(set(clean_js_codes + extract_js_code_with_no_tag(script)))
    if len(r) == 0:
        print(script)
    return r
    

In [11]:
# with open('xss-top500.txt', 'r', encoding='utf-8') as file:
#     scripts = file.readlines()
# print('number of scripts {}'.format(len(scripts)))
# clean_scripts = []
# for sc in scripts:
#     try:
#         #parse the response HTML page
#         clean_scripts.append(extract_js_code(sc.strip()))
#     except:
#         print(sc)
# clean_scripts = list(filter(lambda x: len(x)>0, clean_scripts))
# print('number of clean scripts: {}'.format(len(clean_scripts)))

In [12]:
# dataset = []
# for sc in clean_scripts:
#     for s in sc:
#         try:
#             sequence_of_syntax_units(s)
#             dataset.append(s)
#         except:
#             print(sc)

In [13]:
target = [1] * 181

In [15]:
# len(dataset)

In [16]:
# model, history = pipeline(dataset, target, 0.1)

In [17]:
# outputs = [layer.output for layer in model.layers]
# outputs

In [18]:
# plot_model(model, show_shapes=True)

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Sequential
from keras import layers
from keras.backend import clear_session
from keras.preprocessing.text import Tokenizer
from keras.layers import Activation, Dense, Reshape 
import numpy as np
from keras import backend as K
import matplotlib.pyplot as plt
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout, Conv1D, GlobalMaxPooling1D
from keras.models import load_model
from numpy import asarray
from numpy import save
from numpy import load
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import esprima as esp
import re
from keras.initializers import Constant
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.utils import plot_model
from keras import Model
import pydot
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

plt.style.use('ggplot')

In [37]:
with open("G:/JSContanaDataSet/programs_training.txt", 'r', encoding='utf-8') as file:
    path_scripts = file.readlines()
script_df = pd.DataFrame(columns=["ScriptName","Label"])
scripts = []
Label = []
for path_script in range(len(path_scripts)-1):
    try:
        s_p = "G:/JSContanaDataSet/"+path_scripts[path_script][:-1]
        with open(s_p, 'r', encoding='utf-8') as js_file:
            script = js_file.read()
        scripts.append(script)
        if path_script%2==0:
            Label.append(0)
        else:
            Label.append(1)
    except:
#         print("error =>"+str(path_script)+": "+path_scripts[path_script])
script_df["ScriptName"] = scripts
script_df["Label"] = Label

error =>490: data/BashtonLtd/rrequest/packages/mizzao:sharejs/ace-builds/textarea/src/mode-lua.js

error =>1644: data/oxfordinternetinstitute/InteractiveVis/network/js/sigma/sigma.parseGexf.js

error =>1701: data/BashtonLtd/rrequest/packages/mizzao:sharejs/ace-builds/src/mode-django.js

error =>2013: data/webadeo/git-simpler-better-faster-stronger/node_modules/gulp-imagemin/node_modules/imagemin/node_modules/imagemin-optipng/node_modules/optipng-bin/node_modules/bin-wrapper/node_modules/download/node_modules/request/node_modules/qs/index.js

error =>2281: data/BashtonLtd/rrequest/packages/mizzao:sharejs/ace-builds/src/mode-tcl.js

error =>2323: data/vivin/regula/src/lib/html5Forms/js/jscalendar-1.0/lang/calendar-lt.old.js

error =>2512: data/BashtonLtd/rrequest/packages/mizzao:sharejs/ace-builds/textarea/src/mode-scheme.js

error =>2615: data/BashtonLtd/rrequest/packages/mizzao:sharejs/ace-builds/src-noconflict/mode-jsoniq.js

error =>2859: data/NodeBB/NodeBB/public/vendor/xregexp/unic

error =>32019: data/BashtonLtd/rrequest/packages/mizzao:sharejs/ace-builds/src/mode-assembly_x86.js

error =>32303: data/datejs/Datejs/src/date.js

error =>33141: data/BashtonLtd/rrequest/packages/mizzao:sharejs/ace-builds/src/mode-jade.js

error =>33142: data/payonesmile/formdesign/Formdesign4_1_GBK/formdesign/leipi.formdesign.v4.js

error =>33455: data/BashtonLtd/rrequest/packages/mizzao:sharejs/ace-builds/textarea/src/mode-toml.js

error =>33568: data/pielgrzym/django-minishop/media/js/tiny_mce/plugins/xhtmlxtras/js/element_common.js

error =>34008: data/moravianlibrary/differ/js/sorttable.js

error =>34338: data/webadeo/git-simpler-better-faster-stronger/node_modules/gulp-imagemin/node_modules/imagemin/node_modules/imagemin-optipng/node_modules/optipng-bin/node_modules/bin-wrapper/node_modules/download/node_modules/decompress/node_modules/tar/lib/pack.js

error =>34464: data/jvandemo/generator-jabl/a:directive/templates/directiveSpec.js

error =>34601: data/lifesinger/lifesinger.gi

error =>63625: data/bryansray/tempo/public/javascripts/tiny_mce/plugins/autosave/editor_plugin_src.js

error =>63771: data/jbasdf/muck/public/javascripts/tiny_mce/plugins/advfiletoo/editor_plugin_src.js

error =>64120: data/arcanis/peg.arcanis.fr/static/vendors/angular/CodeMirror.js

error =>64318: data/envadrouillecorp/envadrouille/admin/pages/update/scripts/lang/option_fr.js

error =>64435: data/yinqiao/supercss/3/3.7/zQuery1.js

error =>64481: data/threedubmedia/jquery.threedubmedia/jquery/qunit.js

error =>64532: data/alessioalex/ClientManager/public/js/lib/mocha.js

error =>64946: data/montylounge/django-mingus/mingus/static/mingus/js/tiny_mce/plugins/directionality/editor_plugin_src.js

error =>65108: data/BashtonLtd/rrequest/packages/mizzao:sharejs/ace-builds/textarea/src/theme-tomorrow.js

error =>65380: data/yagopv/BgEngine/BgEngine.Web/Scripts/internal/bg.tagger.js

error =>66140: data/bdecoste/openshift-origin-cartridge-activemq/versions/5.8/apache-activemq/webapps/admin/js/

error =>87379: data/jvandemo/generator-jabl/a:controller/index.js

error =>87620: data/trachalakis/fasttrack/webroot/js/tinymce-dev-3.2.2.3/plugins/tabfocus/editor_plugin_src.js

error =>88249: data/webadeo/git-simpler-better-faster-stronger/node_modules/gulp-imagemin/node_modules/imagemin/node_modules/imagemin-optipng/node_modules/optipng-bin/node_modules/bin-wrapper/node_modules/download/node_modules/decompress/node_modules/tar/node_modules/block-stream/test/pause-resume.js

error =>88333: data/kaelzhang/neuron/test/mod/require/*/require.js

error =>88981: data/webadeo/git-simpler-better-faster-stronger/node_modules/gulp-imagemin/node_modules/imagemin/node_modules/imagemin-optipng/node_modules/optipng-bin/node_modules/bin-wrapper/node_modules/download/node_modules/request/node_modules/oauth-sign/index.js

error =>90299: data/BashtonLtd/rrequest/packages/mizzao:sharejs/ace-builds/textarea/src/mode-batchfile.js

error =>90377: data/PrincetonUSG/USG-srv-dev/tigerapps/static/ttrade/js/fo

In [36]:
script_df = pd.read_csv('G:/JSContanaDataSet/script_path_df.csv')

In [37]:
script_df.head()

,ScriptName,Label
0,/* -*- Mode: C++; tab-width: 2; indent-tabs-mo...,0
1,'use strict';var Promise = require('bluebird')...,1
2,"CKEDITOR.plugins.setLang(""showblocks"",""en"",{to...",0
3,"'use strict';describe('myApp.view1 module', fu...",1
4,"'use strict';angular.module(""ngLocale"", [], [""...",0


In [38]:
def ext(fn):
    fn = re.sub("\\n","",fn)
    fn = re.sub("\\t","",fn)
    return fn

In [39]:
# script_df["ScriptName"] = script_df["ScriptName"].apply(ext)

In [53]:
columns = script_df.columns

In [59]:
columns.shape[0]==2

True

In [76]:
i = 0
script_df["ScriptName"].apply(sequence_of_syntax_units)
# sequence_of_syntax_units(s[1:-1])

1
2
3
4


Error: Line 1: Unexpected end of input

In [90]:
i=0
# for i in range(100):
sequence_of_syntax_units(s[10])

1


'{ type: "Program", sourceType: "script", body: [ { type: "ExpressionStatement", expression: { type: "CallExpression", callee: { type: "Identifier", name: "define" }, arguments: [ { type: "Literal", value: "dojox/editor/plugins/nls/ru/latinEntities", raw: "\\"dojox/editor/plugins/nls/ru/latinEntities\\"" }, { type: "ObjectExpression", properties: [ { type: "Property", key: { type: "Identifier", name: "iexcl" }, computed: False, value: { type: "Literal", value: "\\u0438\\u043d\\u0432\\u0435\\u0440\\u0442\\u0438\\u0440\\u043e\\u0432\\u0430\\u043d\\u043d\\u044b\\u0439 \\u0432\\u043e\\u0441\\u043a\\u043b\\u0438\\u0446\\u0430\\u0442\\u0435\\u043b\\u044c\\u043d\\u044b\\u0439 \\u0437\\u043d\\u0430\\u043a", raw: "\\"\\u0438\\u043d\\u0432\\u0435\\u0440\\u0442\\u0438\\u0440\\u043e\\u0432\\u0430\\u043d\\u043d\\u044b\\u0439 \\u0432\\u043e\\u0441\\u043a\\u043b\\u0438\\u0446\\u0430\\u0442\\u0435\\u043b\\u044c\\u043d\\u044b\\u0439 \\u0437\\u043d\\u0430\\u043a\\"" }, kind: "init", method: False, short

In [42]:
print(script_df["ScriptName"][0])

/* -*- Mode: C++; tab-width: 2; indent-tabs-mode: nil; c-basic-offset: 2 -*- *//* This Source Code Form is subject to the terms of the Mozilla Public * License, v. 2.0. If a copy of the MPL was not distributed with this * file, You can obtain one at http://mozilla.org/MPL/2.0/. */var gTestfile = 'regress-472450-04.js';//-----------------------------------------------------------------------------var BUGNUMBER = 472450;var summary = 'TM: Do not assert: StackBase(fp) + blockDepth == regs.sp';var actual = '';var expect = '';//-----------------------------------------------------------------------------test();//-----------------------------------------------------------------------------function test(){  enterFunc ('test');  printBugNumber(BUGNUMBER);  printStatus (summary);   jit(true);  ({__proto__: #1=[#1#]});  function f(){     eval("for (var y = 0; y < 1; ++y) { for each (let z in [null, function(){}, null, '', null, '', null]) { let x = 1, c = []; } }");   }  f();  jit(false);  repor

In [87]:

target = [1] * 181
test_size = 0.2

# X = sequence_of_syntax_units(list(script_df["ScriptName"]))
    
# # 2- train test split 
# indeices = np.arange(len(script_df["ScriptName"])).astype('int')
# np.random.shuffle(indeices)
# X = np.array(X)
# target = np.array(target)
# X = X[indeices]
# y = target[indeices]
# num_of_test_samples = int(test_size * len(script_df["ScriptName"]))
# X_train, X_test, y_train, y_test = X[:-num_of_test_samples], X[-num_of_test_samples:], y[:-num_of_test_samples], y[-num_of_test_samples:]


# 3- train static word2vec model and save it 
# model, voc_size = train_static_word2vec(X_train)
# save_static_word2vec(model, MODEL_FILENAME, X_train, TRAIN_DATA_FILENAME)

# 4- build nerual network
# nn = neural_network()
# preprocess input

X_train, X_test, y_train, y_test = train_test_split(script_df["ScriptName"],script_df["Label"], test_size=0.25, random_state=1000)

X_train_, X_test_ = preprocessing_dataset(X_train, X_test)
# feed data to nn


In [91]:
X_train_.shape

(74795, 1024)

In [93]:
y_train.shape

(74795,)

In [92]:
X_test_.shape

(24932, 1024)

In [94]:
y_test.shape

(24932,)

In [70]:
with open("G:/JSContanaDataSet/programs_training.txt", 'r', encoding='utf-8') as file:
    path_scripts = file.readlines()
# script_df = pd.DataFrame(columns=["ScriptName","Label"])
s = []
l = []
for path_script in range(100):
    try:
        s_p = "G:/JSContanaDataSet/"+path_scripts[path_script][:-1]
        with open(s_p, 'r', encoding='utf-8') as js_file:
            script = js_file.read()
        s.append(script)
        if path_script%2==0:
            l.append(0)
        else:
            l.append(1)
    except:
        print("error =>"+str(path_script)+": "+path_scripts[path_script])
# script_df["ScriptName"] = scripts
# script_df["Label"] = Label

In [88]:
# import os  
# os.makedirs('folder/subfolder', exist_ok=True)  
script_df.to_csv('G:/JSContanaDataSet/script_path_df.csv',index=False)

In [12]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

In [26]:
train_generator=datagen.flow_from_dataframe(
dataframe=script_df,
directory="G:/JSContanaDataSet/",
x_col="ScriptName",
y_col="Label",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="input",
target_size=(32,32))



valid_generator=datagen.flow_from_dataframe(
dataframe=script_df,
directory="G:/JSContanaDataSet/",
x_col="ScriptName",
y_col="Label",
subset="validation",
batch_size=32,
seed=42,
shuffle=True, # False For Test generator only, for others set True
class_mode="input", # “categorical”(default), ”binary”, ”sparse”, ”input”
target_size=(32,32))



# test_datagen=ImageDataGenerator(rescale=1./255.)
# test_generator=test_datagen.flow_from_dataframe(
# dataframe=testdf,
# directory="./test/",
# x_col="id",
# y_col=None,
# batch_size=32,
# seed=42,
# shuffle=False,
# class_mode=None,
# target_size=(32,32))

C:\ProgramData\Anaconda3\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:282: UserWarning: Found 99727 invalid image filename(s) in x_col="ScriptName". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 0 validated image filenames.
Found 0 validated image filenames.


In [25]:
print(train_generator)

In [13]:
print(len(scripts))
scripts[0]

99727


'data/mozilla/rhino/testsrc/tests/js1_8/extensions/regress-472450-04.js'